In [3]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

In [4]:
from dash import Dash, html, Output, Input
import dash_leaflet as dl
import dash_leaflet.express as dlx
from dash_extensions.enrich import html, DashProxy
from dash_extensions.javascript import arrow_function, assign
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
# standalone map
import folium
from folium import plugins
# data manipulation
import numpy as np
import pandas as pd
import json
import geojson
import geopandas as gpd

data_directory = '/home/jan/Uni/DS-Project/data/Maps/'

/tmp/ipykernel_48082/3046874679.py:17: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


# Design

In [5]:
# specify design choices
app = Dash(external_stylesheets=[dbc.themes.LITERA])

# create 3 pages
layout_page_1 = html.Div([dbc.Container([dbc.Row([
                        dbc.Col(html.H1("Page 1"), width={"size": 6, "offset": 3})
                    ])])])

layout_page_2 = html.Div([dbc.Container([dbc.Row([
                        dbc.Col(html.H1("Page 2"), width={"size": 6, "offset": 3})
                    ])])])

layout_page_3 = html.Div([dbc.Container([dbc.Row([
                        dbc.Col(html.H1("Page 3"), width={"size": 6, "offset": 3})
                    ])])])

# callback for switching between pages
@app.callback(
    Output("page-content", "children"),
    [Input("page-selector", "value")]
)
def display_page(page_idx):
    if page_idx == "page-1":
        return layout_page_1
    elif page_idx == "page-2":
        return layout_page_2
    elif page_idx == "page-3":
        return layout_page_3

# layout for navbar
app.layout = html.Div([dbc.Container(
    [dbc.Row([dbc.Col([
        html.H1("Dashboard with Bootstrap"),
            dbc.RadioItems(
                id="page-selector",
                options=[
                    {"label": "Page 1", "value": "page-1"},
                    {"label": "Page 2", "value": "page-2"},
                    {"label": "Page 3", "value": "page-3"},
                    ],
                value="page-1",
                inline=True,),],
                width={"size": 2, "offset": 5},),]),
    dbc.Row([dbc.Col(
                id="page-content",
                width={"size": 8, "offset": 2},),]),]),])


# Map

In [6]:
# load germany maps & polygons
#bund = gpd.read_file(data_directory + 'Bundeskarte.geojson').to_crs(4326)
land = gpd.read_file(data_directory + 'Laenderkarte.geojson').to_crs(4326)
kreis = gpd.read_file(data_directory + 'Kreiskarte.geojson').to_crs(4326)
polygon = gpd.read_file(data_directory + 'brandenburg_polygons.geojson').set_crs(25833, allow_override = True).to_crs(4326)

In [7]:
polygon_offset = polygon.copy().to_crs(3043)
polygon_offset['geometry'] = polygon_offset.geometry.buffer(-12, single_sided = True)
polygon_offset['area'] = polygon_offset['geometry'].area / 10**3
polygon_offset['geometry'] = polygon_offset.geometry.to_crs(4326)

In [8]:
# add random score between 0 and 10 to kreis
np.random.seed(123)
kreis.insert(
    13,
    "Score",
    np.random.uniform(0, 11, size=len(kreis))
)

In [9]:
'''
with open("kreis.json", "w") as f:
    f.write(kreis.to_json())
with open("land.json", "w") as f:
    f.write(land.to_json())
with open("polygone.json", "w") as f:
    f.write(polygon_offset.to_json())
'''

'\nwith open("kreis.json", "w") as f:\n    f.write(kreis.to_json())\nwith open("land.json", "w") as f:\n    f.write(land.to_json())\nwith open("polygone.json", "w") as f:\n    f.write(polygon_offset.to_json())\n'

In [10]:
# feed kreis in dash app
kreis_path = "assets/kreis.json"
# load kreis
with open(kreis_path, 'r') as f:
    kreis = [f['properties'] for f in json.loads(f.read())['features']]

# feed land in dash app
land_path = "assets/land.json"
# load kreis
with open(land_path, 'r') as f:
    land = [f['properties'] for f in json.loads(f.read())['features']]

# feed polygone in dash app
polygon_path = "assets/polygone.json"
# load kreis
with open(polygon_path, 'r') as f:
    polygone = [f['properties'] for f in json.loads(f.read())['features']]

In [11]:
# define info box
def get_info(feature=None):
    header = [html.H4("Fitness für Ohren")]
    if not feature:
        return header + [html.P("Über Provinz hovern")]
    return header + [html.B(feature["properties"]["NAME_3"]), html.Br(),
                     "{:.2f} Lustnauer Score".format(feature["properties"]["Score"])]

classes = [0, 2, 4, 6, 8, 10]
colorscale = ['#ffd34c', '#e5f3e5', '#99d099', '#66b967', '#32a234', '#008b02']
style = dict(weight=2, opacity=1, color='white', dashArray='3', fillOpacity=0.7)
# Create colorbar.
ctg = ["{}+".format(cls, classes[i + 1]) for i, cls in enumerate(classes[:-1])]
colorbar = dlx.categorical_colorbar(categories=ctg, colorscale=colorscale, width=300, height=30, position="bottomleft")
# Geojson rendering logic, must be JavaScript as it is executed in clientside.
style_handle = assign("""function(feature, context){
    const {classes, colorscale, style, colorProp} = context.props.hideout;  // get props from hideout
    const value = feature.properties[colorProp];  // get value the determines the color
    for (let i = 0; i < classes.length; ++i) {
        if (value > classes[i]) {
            style.fillColor = colorscale[i];  // set the fill color according to the class
        }
    }
    return style;
}""")
# Create geojsons.
kreis_geojson = dl.GeoJSON(url=f"/{kreis_path}",  # url to geojson file
                     options=dict(style=style_handle),  # how to style each polygon
                     zoomToBounds=True,  # when true, zooms to bounds when data changes (e.g. on load)
                     zoomToBoundsOnClick=True,  # when true, zooms to bounds of feature (e.g. polygon) on click
                     hoverStyle=arrow_function(dict(weight=5, color='#666', dashArray='')),  # style applied on hover
                     hideout=dict(colorscale=colorscale, classes=classes, style=style, colorProp="Score"),
                     id="geojson")
polygon_geojson = dl.GeoJSON(url=f"/{polygon_path}",  # url to geojson file
                     #options=dict(style=style_handle),  # how to style each polygon
                     #zoomToBounds=True,  # when true, zooms to bounds when data changes (e.g. on load)
                     zoomToBoundsOnClick=True,  # when true, zooms to bounds of feature (e.g. polygon) on click
                     #hoverStyle=arrow_function(dict(weight=5, color='#666', dashArray='')),  # style applied on hover
                     #hideout=dict(colorscale=colorscale, classes=classes, style=style, colorProp="Score"),
                     id="geojson")


# Create info control.
info = html.Div(children=get_info(), id="info", className="info",
                style={"position": "absolute", "top": "10px", "right": "50px", "z-index": "1000"})
# app backend
app = Dash(prevent_initial_callbacks=True)
'''
app.layout = html.Div(
    # show map
    [dl.Map(
        children=[#dl.LayersControl(
                  #    dl.BaseLayer(
                          dl.TileLayer(),
                  #        ),
                      #dl.Overlay(dl.LayerGroup(polygon_geojson), name = 'Ohren', checked = True)
                  #  ),
                  dl.GestureHandling(), kreis_geojson, colorbar, info
                  ],
    # set max bounds and zoom
    maxBounds = [
        # north-east
        [55.5, 15.7],
        # south-west
        [47.1, 5.4]
        ],
    zoom = 6,
    minZoom = 6,
    # set style in dash board                  
    style={'width': '50%', 'height': '75vh', 'margin': "auto", "display": "block"})],
    id="map")
'''
app.layout = html.Div([dl.Map(children=[dl.TileLayer(), kreis_geojson, colorbar, info])],
                      style={'width': '50%', 'height': '75vh', 'margin': "auto", "display": "block"}, id="map")

@app.callback(
    Output("info", "children"),
    [Input("geojson", "hover_feature")]
)

def info_hover(feature):
    return get_info(feature)

# Run App

In [12]:
# run app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [03/Feb/2023 14:46:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2023 14:46:39] "GET /_dash-component-suites/dash/deps/react@16.v2_7_1m1672739997.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2023 14:46:39] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_7_1m1672739997.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2023 14:46:39] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_7_1m1672739997.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2023 14:46:39] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_7_1m1672739997.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2023 14:46:39] "GET /_dash-component-suites/dash_extensions/dash_extensions.v0_1_9m1672739997.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2023 14:46:39] "GET /_dash-component-suites/dash_leaflet/dash_leaflet.v0_1_23m1672739996.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2023 14:46:39] "GET /_dash-component-suites/dash/dcc/d